<a href="https://colab.research.google.com/github/DarshanSajeev/COMP2611/blob/main/Copy_of_SearchExercise5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py
from bbmodcache.bbSearch import SearchProblem, search

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18767  100 18767    0     0   389k      0 --:--:-- --:--:-- --:--:--  389k


In [30]:
# def cost(path, state):
#         return len(path)

# small_cost = search( RW_PROBLEM, 'BF/FIFO', 30000, cost=cost, loop_check=True, return_info=True)

# def manhattan_heuristic(state):
#     goal_r, goal_c = len(state.blockstate) - 1, len(state.blockstate[0]) - 1  # Bottom-right corner
    
#     total = 0
#     for row in range(state.nrows):
#         for col in range(state.ncols):
#             if state.blockstate[row][col] == 1:  # Find red block tiles
#                 total += abs(goal_r - row) + abs(goal_c - col)
    
#     return total

# small_manhattan = search( RW_PROBLEM, 'DF/LIFO', 30000, loop_check=True, randomise=True, return_info=True, cost=cost, heuristic=manhattan_heuristic)

In [31]:
class Robot:
    def __init__(self, location, carried_items=None):
        if carried_items is None:
            carried_items = []  # Default to an empty list if no items are specified
        self.location = location
        self.carried_items = carried_items

    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location, self.carried_items ) )

class Door:
    def __init__(self, roomA, roomB, doorkey=None, locked=False):
        self.goes_between = {roomA, roomB}
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}

    ## Define a unique string representation for a door object
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )
    
class State:
    def __init__(self, robot, doors, room_contents):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents  # Ensure this attribute is passed and initialized

    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )

In [32]:
ROOM_CONTENTS = {
    'Storage' : {'M3','P1','H2','E2'},
    'Office' :{'keycard'},
    'English' : {'E1', 'E3'},
    'Maths' : {'M1', 'M2'},
    'History' : {'H1'},
    'Physics' : {'P2'},
}


# Define the rooms that need to be unlocked and locked
storage_rooms = ['History', 'English', 'Maths', 'Physics']
subject_rooms = ['History', 'English', 'Maths', 'Physics']

# Initialize the doors list with the locked rooms (History and Physics)
# Initialize the doors list with the locked rooms (History and Physics)
DOORS = [
    # History and Physics are locked and need the keycard to access
    Door('Storage', 'Physics', doorkey='keycard', locked=True),
    Door('Storage', 'History', doorkey='keycard', locked=True),
    Door('Storage', 'Office', doorkey='keycard', locked=False)
]

# Add doors for Storage to all other rooms (unlocked), except History and Physics
for room in storage_rooms:
    if room not in ['History', 'Physics']:  # Exclude History and Physics
        DOORS.append(Door('Storage', room, locked=False))

# Output the doors to verify
for door in DOORS:
    print(door)



('door', {'Physics', 'Storage'}, 'keycard', True)
('door', {'History', 'Storage'}, 'keycard', True)
('door', {'Office', 'Storage'}, 'keycard', False)
('door', {'English', 'Storage'}, None, False)
('door', {'Maths', 'Storage'}, None, False)


In [33]:
from copy import deepcopy

class RobotWorker(SearchProblem):

    def __init__(self, state, goal_item_locations):
        self.initial_state = state
        self.goal_item_locations = goal_item_locations

    def possible_actions(self, state):
        robot_location = state.robot.location
        actions = []

        # Can put down any carried item if it's the correct room
        for item in state.robot.carried_items:
            # Check if the item belongs to the current room
            if item in self.goal_item_locations.get(robot_location, set()):
                actions.append(("put down", item))

        # Can pick up any item in the current room
        for item in state.room_contents[robot_location]:
            # If the item needs to be sorted, it can be picked up
            if item not in state.robot.carried_items:
                actions.append(("pick up", item))

        # If there is an unlocked door between robot location and another location, robot can move
        for door in state.doors:
            if not door.locked and robot_location in door.goes_between:
                actions.append(("move to", door.other_loc[robot_location]))

        # Return the list of possible actions
        return actions

    def successor(self, state, action):
        next_state = deepcopy(state)
        act, target = action

        if act == "put down":
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)
        
        if act == "pick up":
            next_state.robot.carried_items.append(target)
            next_state.room_contents[state.robot.location].remove(target)

        if act == "move to":
            next_state.robot.location = target

        return next_state

    def goal_test(self, state):
        for room, contents in self.goal_item_locations.items():
            for item in contents:
                if item not in state.room_contents[room]:
                    return False
        return True

    def display_state(self, state):
        print("Robot location:", state.robot.location)
        print("Robot carrying:", state.robot.carried_items)
        print("Room contents:", state.room_contents)


In [34]:
# Initial State Setup
initial_robot = Robot('Storage')
initial_state = State(initial_robot, DOORS, ROOM_CONTENTS)

# Goal Locations Setup
goal_item_locations = {
    'Storage': {'M2', 'H1', 'D1', 'E2'},
    'Maths': {'M3'},
    'English': {'E3'},
    'History': {'H2'},
    'Physics': {'P2'},
    'Business': {'B2'},
}

# Instantiate the RobotWorker with the initial state and goal
RW_PROBLEM = RobotWorker(initial_state, goal_item_locations)

poss_acts = RW_PROBLEM.possible_actions( RW_PROBLEM.initial_state )
poss_acts

for act in poss_acts:
    print("Action", act, "leads to the following state:")
    next_state = RW_PROBLEM.successor( RW_PROBLEM.initial_state, act )
    RW_PROBLEM.display_state(next_state)
    print()

Action ('pick up', 'M3') leads to the following state:
Robot location: Storage
Robot carrying: ['M3']
Room contents: {'Storage': {'P1', 'E2', 'H2'}, 'Office': {'keycard'}, 'English': {'E1', 'E3'}, 'Maths': {'M2', 'M1'}, 'History': {'H1'}, 'Physics': {'P2'}}

Action ('pick up', 'H2') leads to the following state:
Robot location: Storage
Robot carrying: ['H2']
Room contents: {'Storage': {'M3', 'P1', 'E2'}, 'Office': {'keycard'}, 'English': {'E1', 'E3'}, 'Maths': {'M2', 'M1'}, 'History': {'H1'}, 'Physics': {'P2'}}

Action ('pick up', 'E2') leads to the following state:
Robot location: Storage
Robot carrying: ['E2']
Room contents: {'Storage': {'M3', 'P1', 'H2'}, 'Office': {'keycard'}, 'English': {'E1', 'E3'}, 'Maths': {'M2', 'M1'}, 'History': {'H1'}, 'Physics': {'P2'}}

Action ('pick up', 'P1') leads to the following state:
Robot location: Storage
Robot carrying: ['P1']
Room contents: {'Storage': {'M3', 'E2', 'H2'}, 'Office': {'keycard'}, 'English': {'E1', 'E3'}, 'Maths': {'M2', 'M1'}, 'Hi

In [ ]:
search( RW_PROBLEM, 'BF/FIFO', 1000000, loop_check=True, return_info=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 1000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
.................................................................................................... (200000)
.................................................................................................... (300000)
..................................................................................